## Mini-AES complete with 2 rounds

## Import Libraries

In [1]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00


In [2]:
!pip install qiskit-aer
!pip install pylatexenc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=a62a7b8175e256c97d1d8abcf0fa3118f4ca5d08d5939492cf3d2ac8655b5f3a
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import *
from qiskit.circuit.library import *
from qiskit.visualization import circuit_drawer
from qiskit import *
from qiskit.circuit.library import *
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import process_fidelity

## Defining Functions for Encoding

In [4]:
def Plain_text(qc, P):
    for i in range(len(P)):
        if int(P[i]) == 1:
            qc.x(i)
    return qc
def Key_addition_0(qc, K):
    for i in range(len(K)):
        if int(K[i]) == 1:
            qc.x(i)
    return qc

def Key_addition_1(qc, start, end, K1):
    for i in range(start,end):
        if int(K1[i-start]) == 1:
            qc.x(i)
    return qc
def S_Box_0bit(qc):
    qc.ccx(0,1,4)
    qc.ccx(2,4,5)
    qc.cx(4,5)
    qc.reset(4)
    qc.ccx(1,2,4)
    qc.ccx(3,4,5)
    qc.cx(4,5)
    qc.cx([0,1,3],[5,5,5])
    qc.x(5)
    return qc
def S_Box_1bit(qc):
    qc.ccx(0,2,4)
    qc.ccx(3,4,5)
    qc.ccx(1,3,5)
    qc.ccx(2,3,5)
    qc.cx([0,1,4],[5,5,5])
    qc.x(5)
    return qc
def S_Box_2bit(qc):
    qc.ccx(0,1,4)
    qc.ccx(2,4,5)
    qc.ccx(3,4,5)
    qc.ccx(0,1,5)
    qc.ccx(0,2,5)
    qc.ccx(0,3,5)
    qc.ccx(1,2,5)
    qc.ccx(1,3,5)
    qc.ccx(2,3,5)
    qc.cx([3,2],[5,5])
    qc.x(5)
    return qc
def S_Box_3bit(qc):
    qc.ccx(0,3,4)
    qc.ccx(2,4,5)
    qc.ccx(1,3,5)
    qc.cx([2,0,4],[5,5,5])
    return qc
def Row_Shift(qc):
    # Applying Row Shift
    qc.swap(12,20)
    qc.swap(13,21)
    qc.swap(14,22)
    qc.swap(15,23)
    return qc
def Mix_Column(qc):
    #copy f and 1 to 0-3, 4-7
    qc.cx([8,9,10,11],[0,1,2,3])
    qc.cx([12,13,14,15],[4,5,6,7])
    qc.barrier()
    qc.reset([12,13,14,15])
    qc.barrier()
    #copy 7 to 12-15
    qc.cx([20,21,22,23],[12,13,14,15])
    qc.barrier()
    ##3(f)
    qc.cx(0,3)
    qc.cx(1,0)
    qc.cx(2,1)
    qc.cx(3,2)
    ##2(1)
    qc.swap(4,7)
    qc.swap(5,4)
    qc.swap(5,6)
    qc.cx(7,6)
    ##XOR  (3(f)+2(1))
    qc.cx(4,0)
    qc.cx(5,1)
    qc.cx(6,2)
    qc.cx(7,3)
    ##inverse 2(1)
    qc.cx(7,6)
    qc.swap(5,6)
    qc.swap(5,4)
    qc.swap(4,7)
    ########################################
    ##3(7)
    qc.cx(12,15)
    qc.cx(13,12)
    qc.cx(14,13)
    qc.cx(15,14)
    ##2(a)
    qc.swap(16,19)
    qc.swap(17,16)
    qc.swap(17,18)
    qc.cx(19,18)
    ##XOR (2(a) + 3(7))
    qc.cx(16,12)
    qc.cx(17,13)
    qc.cx(18,14)
    qc.cx(19,15)
    ## inverse 2(a)
    ##2(a)
    qc.cx(19,18)
    qc.swap(17,18)
    qc.swap(17,16)
    qc.swap(16,19)
    ########################################
    qc.barrier()
    ########################################
    ## 3 on inverse 2(1) ---> 3(1)
    qc.cx(4,7)
    qc.cx(5,4)
    qc.cx(6,5)
    qc.cx(7,6)
    ## 3 on inverse 2(a) --> 2(a)
    qc.cx(16,19)
    qc.cx(17,16)
    qc.cx(18,17)
    qc.cx(19,18)
    ######################################
    # 2f
    qc.swap(8,11)
    qc.swap(8,9)
    qc.swap(9,10)
    qc.cx(11,10)
    # 2(7)
    qc.swap(20,23)
    qc.swap(20,21)
    qc.swap(21,22)
    qc.cx(23,22)
    #XOR (2(f)+3(1))
    qc.cx([8,9,10,11],[4,5,6,7])
    #XOR (3(a)+2(7))
    qc.cx([20,21,22,23],[16,17,18,19])
    return qc
# ROUND 2
def S_box_round2_0bit(qc):
    qc.ccx(0,1,4)
    qc.ccx(2,4,5)
    qc.cx(4,5)
    qc.reset(4)
    qc.ccx(1,2,4)
    qc.ccx(3,4,5)
    qc.cx(4,5)
    qc.cx([0,1,3],[5,5,5])
    qc.x(5)
    return qc
def S_box_round2_1bit(qc):
    qc.ccx(0,2,4)
    qc.ccx(3,4,5)
    qc.ccx(1,3,5)
    qc.ccx(2,3,5)
    qc.cx([0,1,4],[5,5,5])
    qc.x(5)
    return qc
def S_box_round2_2bit(qc):
    qc.ccx(0,1,4)
    qc.ccx(2,4,5)
    qc.ccx(3,4,5)
    qc.ccx(0,1,5)
    qc.ccx(0,2,5)
    qc.ccx(0,3,5)
    qc.ccx(1,2,5)
    qc.ccx(1,3,5)
    qc.ccx(2,3,5)
    qc.cx([3,2],[5,5])
    qc.x(5)
    return qc
def S_box_round2_3bit(qc):
    qc.ccx(0,3,4)
    qc.ccx(2,4,5)
    qc.ccx(1,3,5)
    qc.cx([2,0,4],[5,5,5])
    return qc
def Row_Shift_round2(qc):
  qc.swap([10,11,12,13],[18,19,20,21])
  return qc
def Key_addition_round2(qc, start, end, K2):
    for i in range(start,end):
        if int(K2[i-start]) == 1:
            qc.x(i)
    return qc



## Mini-AES

In [5]:
qc = QuantumCircuit(28,16)
P = ['1001', '1100', '0110', '0011']
K = ['1100', '0011', '1111', '0000']
s=8
t=6
l=16
for i in range(4):
    Plain_text(qc, P[i])
    qc.barrier()
    Key_addition_0(qc, K[i])
    qc.barrier()
    S_Box_0bit(qc)
    qc.cx(5,s)
    qc.barrier()
    qc.reset([4,5])
    S_Box_1bit(qc)
    qc.cx(5,s+1)
    qc.barrier()
    qc.reset([4,5])
    S_Box_2bit(qc)
    qc.cx(5,s+2)
    qc.barrier()
    qc.reset([4,5])
    S_Box_3bit(qc)
    qc.cx(5,s+3)
    qc.barrier()
    qc.reset(list(range(0,8)))
    s= s+4

Row_Shift(qc)
qc.barrier()
Mix_Column(qc)
qc.barrier()
#####################################
qc.reset([8,9,10,11])
qc.cx([16,17,18,19],[8,9,10,11])
#####################################
qc.barrier()
k1 = '0011000011111111'
Key_addition_1(qc,0,16, k1)
qc.reset(list(range(16,28)))
qc.barrier()
#####################################
## copy  except 1st to below 12
qc.cx([4,5,6,7, 8,9,10,11, 12,13,14,15],[16,17,18,19, 20,21,22,23,  24,25,26,27])
qc.barrier()
qc.reset([4,5,6,7, 8,9,10,11, 12,13,14,15])
# ROUND 2
for i in range(4):
    S_box_round2_0bit(qc)
    qc.cx(5,t)
    qc.barrier()
    qc.reset([4,5])
    S_box_round2_1bit(qc)
    qc.cx(5,t+1)
    qc.barrier()
    qc.reset([4,5])
    S_box_round2_2bit(qc)
    qc.cx(5,t+2)
    qc.barrier()
    qc.reset([4,5])
    S_box_round2_3bit(qc)
    qc.cx(5,t+3)
    qc.barrier()
    qc.reset(list(range(0,8)))
    t= t+4
    if (i==0):
        #copy 2nd state
        qc.cx([16,17,18,19],[0,1,2,3])
        qc.reset([16,17,18,19])
    if (i==1):
        #copy 3rd state
        qc.cx([20,21,22,23],[0,1,2,3])
        qc.reset([20,21,22,23])
    if (i==2):
        #copy 4th state
        qc.cx([24,25,26,27],[0,1,2,3])
        qc.reset([24,25,26,27])
qc.barrier()
Row_Shift_round2(qc)
qc.barrier()
k2 = '0110011010010110'
Key_addition_round2(qc,6,21, k2)
qc.barrier()
qc.measure([6,7,8,9, 10,11,12,13, 14,15,16,17, 18,19,20,21], [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

## Graphical Output

In [6]:
qc.draw('mpl', plot_barriers=True, fold=99999)#.savefig('28bit-Mini')

## Simulation

In [7]:
simulator = AerSimulator(method='automatic', precision='single')
job = simulator.run(qc, shots=1)
result = job.result()
counts = result.get_counts(qc)
print(counts)
#0111 0010 1100 0110

{'0110001101001110': 1}


## Flipping Output

In [8]:
flipped_counts = {count[::-1]: value for count, value in counts.items()}
print("Flipped counts:", flipped_counts)

Flipped counts: {'0111001011000110': 1}
